In [ ]:
!conda create -n yolo11 python=3.12 -y
!conda activate yolo11


CondaError: Run 'conda init' before 'conda activate'



In [ ]:
!uv pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu #CPU
!pip install roboflow supervision ultralytics

In [1]:
# test
!yolo task=detect mode=predict model=yolo11n.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=False

Ultralytics 8.3.181 🚀 Python-3.12.11 torch-2.8.0+cpu CPU (Intel Core(TM) i5-8250U 1.60GHz)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

image 1/1 /home/jenda/Documents/WORKSHOP_AI_vision/notebooks/dog.jpeg: 640x384 2 persons, 1 car, 1 dog, 1 handbag, 122.9ms
Speed: 8.2ms preprocess, 122.9ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 384)
💡 Learn more at https://docs.ultralytics.com/modes/predict


📦 2. Dataset – stáhni a připrav
🐤 Použij Roboflow (zdarma)
- Vytvoř si účet na Roboflow.
- Nahraj nebo vyber dataset (např. COCO, vlastní dataset).
- Exportuj ve formátu YOLOv11
- Zkopíruj svůj API klíč.

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="XXX")
project = rf.workspace("tvé-workspace").project("název-projektu")
dataset = project.version(1).download("yolov11")

### struktura datasetu
datasets/
└── my_dataset/
    ├── images/
    │   ├── train/
    │   ├── val/
    └── labels/
        ├── train/
        ├── val/
    └── data.yaml

In [ ]:
# Odstraň poslední řádky z data.yaml (kompatibilita)
!sed -i '$d' {cesta_k_datasetu}/data.yaml

In [ ]:
!yolo task=detect model=train model=yolov11n.pt data=data.yaml epochs=30 imgsz=640 batch=16 lr0=0.005 lrf=0.0005 optimizer=AdamW cos_lr=True warmup_epochs=5 warmup_momentum=0.8 warmup_bias_lr=0.1 hsv_h=0.015 hsv_s=0.7 hsv_v=0.4 degrees=5 translate=0.1 scale=0.5 shear=0.0 perspective=0.0 flipud=0.2 fliplr=0.5 mosaic=1.0 mixup=0.2 copy_paste=0.1 patience=5 cache=True rect=True workers=1 pretrained=True

In [ ]:
!yolo task=detect \
  mode=val \
  model=runs/detect/train/weights/best.pt \
  data=datasets/my_dataset/data.yaml \
  imgsz=640


In [ ]:
!yolo task=detect \
  mode=predict \
  model=runs/detect/train/weights/best.pt \
  source=path/to/image_or_folder \
  imgsz=640 \
  conf=0.25